In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [ ]:
def filter_movies():
    movie_df = pd.read_csv('../datasets/tmdb_5000_movies.csv')
    credits_df = pd.read_csv('../datasets/tmdb_5000_credits.csv')
    credits_df.drop('title', inplace=True, axis=1)
    df = pd.merge(movie_df, credits_df, left_on='id', right_on='movie_id')
    df = df[df.status == 'Released']
    df.dropna(subset=['overview', 'genres', 'keywords'], inplace=True)
    df.loc[:, ['overview', 'genres', 'keywords']] = df[['overview', 'genres', 'keywords']].fillna('')
    return df
    
df_movies = filter_movies()
print("Количество оставшихся фильмов:", len(df_movies))

## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [ ]:
def calculate_tf_matrix(data):
    data['text'] = data['overview'] + ' ' + data['keywords']
    tf = TfidfVectorizer(stop_words='english', max_features=10000)
    tfidf_matrix = tf.fit_transform(data['text'])
    
    return tfidf_matrix

matrix = calculate_tf_matrix(df_movies)
print("Размер матрицы Tf-Idf:", matrix.shape)

## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

In [ ]:
def calculate_cosine_similarity(matrix, df_movies, assets_folder):
    # Рассчитываем cosine similarity
    cosine_sim = cosine_similarity(matrix, matrix)
    df_cosine_sim = pd.DataFrame(cosine_sim, columns=df_movies['movie_id'], index=df_movies['movie_id'])
    
    # Проверяем наличие файла distance.csv
    distance_filepath = os.path.join(assets_folder, 'distance.csv')
    if not os.path.exists(distance_filepath):
        # Сохраняем DataFrame с расстояниями в CSV-файл
        df_cosine_sim.to_csv(distance_filepath)
    else:
        # Перезаписываем файл, если он уже существует
        df_cosine_sim.to_csv(distance_filepath, index=True)

    # Проверяем наличие файла movies.csv
    movies_filepath = os.path.join(assets_folder, 'movies.csv')
    if not os.path.exists(movies_filepath):
        # Сохраняем объединенный датасет с фильмами в CSV-файл
        df_movies.to_csv(movies_filepath, index=True)
    else:
        # Перезаписываем файл, если он уже существует
        df_movies.to_csv(movies_filepath, index=True)


assets_folder = '../src/assets'
calculate_cosine_similarity(matrix, df_movies, assets_folder)